In [10]:
import torch
import torch.nn as nn
import os
import numpy as np
from string import punctuation

In [11]:
char_to_int = {"'": 1, ',': 2, 'e': 3, 'a': 4, 'r': 5, 'i': 6, 's': 7, 'n': 8, 'o': 9, 't': 10, 'l': 11, 'c': 12, 'd': 13, 'm': 14, 'u': 15, 'h': 16, 'g': 17, 'p': 18, 'b': 19, 'k': 20, 'y': 21, '"': 22, 'f': 23, 'w': 24, 'v': 25, 'z': 26, 'j': 27, 'x': 28, 'q': 29, '-': 30, '.': 31, '[': 32, '1': 33, ']': 34}

In [12]:
train_on_gpu = torch.cuda.is_available()
print(train_on_gpu)

False


In [13]:
def load_model(filename):
    save_filename = os.path.splitext(os.path.basename(filename))[0] + '.pt'
    return torch.load(save_filename)

def pad_words(words, length):
  features = np.zeros((len(words), length), dtype=int)
  for i, row in enumerate(words):
    features[i, -len(row):] = np.array(row)[:length]
  return features

def tokenize_word(words):
    test_ints = []
    for word in words:
        word = word.lower()
        test_ints.append([char_to_int[char] for char in word])
    test_ints = pad_words(test_ints, 28)
    return test_ints


def predict_words(net, words):
    net.eval()

    test_ints = tokenize_word(words)

    features = np.array(test_ints)

    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    h = net.init_hidden(batch_size)

    if (train_on_gpu):
        feature_tensor = feature_tensor.cuda()

    # get the output from the model
    output, h = net(feature_tensor, h)

    pred = torch.round(output.squeeze() / 1000)
    return pred.cpu().data.numpy()

def predict_text(net):
    print('Enter your paragraph: ')
    text = str(input())
    text = ''.join([c for c in text if c not in punctuation])
    words = text.split(' ')
    return predict_words(net, words), np.sum(predict_words(net, words))

In [14]:
class LSTM(nn.Module):


    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        """
        Initialize the model by setting up the layers.
        """
        super(LSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        #dropout layers
        self.dropout = nn.Dropout(0.2)
        # linear layers
        self.fc1 = nn.Linear(hidden_dim, 256)
        self.fc2 = nn.Linear(256, output_size)

        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        # fully-connected layer
        out = nn.functional.relu(self.fc1(lstm_out))
        out = self.dropout(out)
        out = self.fc2(out)
        
        # reshape to be batch_size first
        out = out.view(batch_size, -1)
        out = out[:, -1] # get last batch of labels
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [15]:
net_lstm = torch.load('./trained_lstm_phase1.pt', map_location=torch.device("cpu"))

In [16]:
predict_text(net_lstm)

Enter your paragraph: 
My name is Son. I am a Denisonian!


(array([1., 1., 1., 1., 1., 1., 1., 5.], dtype=float32), 12.0)

In [ ]:
predict_text(net_lstm)

Enter your paragraph: 


In [ ]:
predict_text(net_lstm)